# Visualize Networkss

In [25]:
# Mathematical and Data Managment
import numpy as np
import pandas as pd
import scipy.sparse as sp

# Graph Managment
import graph_tool.all as gt
import networkx as nx
import Functions as Fun
import Segregation as Seg

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import cython
import Cython
from forceatlas2 import forceatlas2_networkx_layout

# Miscellaneous
from glob import glob
from tqdm import tqdm
import random
import os
import time
import multiprocessing as mp
import concurrent.futures

# Reload Utils
import importlib
importlib.reload(Seg)
importlib.reload(Fun)

<module 'Functions' from '/mnt/disk2/fcastrillon/Analysis-of-Tweets-During-the-2021-Social-Unrest/Code/Functions.py'>

In [21]:
ls = []
files = glob('/mnt/disk2/Data/3_Day_Graphs/*.graphml')

for file in files:
    g = gt.load_graph(file)
    N = g.num_vertices()
    ls.append((file,N))

sorted_ls = sorted(ls, key=lambda x: x[1], reverse=True)
sorted_files = [g[0] for g in sorted_ls]

In [22]:
sub_files = sorted_files[0:16]

In [17]:
# PASO 0
def initial_pos():
    master_id = pd.read_csv('/mnt/disk2/Data/3_Day_Graphs/Nodes.csv', sep = ';')
    pos = {i: (random.random(), random.random()) for i in master_id['Idx in Graph']}
    return pos

# PASO 1
def read_graph(file):
    g = gt.load_graph(file)  
    date = file.split('/')[-1].split('.')[0].split('_')[-1]    
    #sub = Fun.fraction_graph(g, fraction, seed = 2)
    g_nx = Fun.to_networkx(g)
    g_nx = g_nx.to_undirected()
    return g_nx, date

# PASO 2.1
def colors_list(g_nx):
    nodes = g_nx.nodes(data=True)
    colors = [node[1]['Color'] for node in nodes]
    return colors

# PASO 2.2
def node_size(g_nx):
    node_degree = [g_nx.degree(node) for node in g_nx.nodes]
    return node_degree

# PASO 2.3
def self_loops(g_nx):
    self_loops = list(nx.selfloop_edges(g_nx))
    non_self_loop_edges = [(u, v) for u, v in g_nx.edges() if (u, v) not in self_loops and (v, u) not in self_loops]
    return non_self_loop_edges

# PASO 3
def layout(g_nx, pos):
    fa2_layout = forceatlas2_networkx_layout(g_nx, pos, niter=1000)
    return fa2_layout

# PASO 4
def draw(g_nx, fa2_layout, node_degree, colors_list, non_self_loop_edges, file):
    fig, ax = plt.subplots(figsize=(18, 12))

    nx.draw_networkx_nodes(g_nx,
                                fa2_layout,
                                node_size = node_degree,
                                node_color = colors_list,
                                edgecolors=(0,0,0,0.1),
                                ax = ax)

    #nx.draw_networkx_edges(g_nx, fa2_layout, edgelist=non_self_loop_edges, alpha=0.1, ax=ax)
    plt.axis('off')
    filename = f'starting_{file}'
    plt.savefig(f'../Results/3_Day_graphs_Viz/{filename}.png')
    plt.close()
    return filename

In [18]:
if __name__ == '__main__':

    start = time.perf_counter()
    
    # PASO 0
    pos = initial_pos()
            
    def Program(file):
        # PASO 1
        g_nx, date = read_graph(file)
            
        # PASO 2
        colors = colors_list(g_nx)
        size = node_size(g_nx)
        non_self_loop_edges = self_loops(g_nx)
            
        # PASO 3 YUCA
        fa2_layout = layout(g_nx, pos)
            
        # PASO 4
        filename = draw(g_nx, fa2_layout, size, colors, non_self_loop_edges, date)
        print(f'Done with {filename}')
        return fa2_layout
    
    with concurrent.futures.ProcessPoolExecutor() as executor:
        executor.map(Program, sub_files)

    finish = time.perf_counter()
    
    print(f'Finished in {round(finish-start, 2)} second(s)')

Done with starting_2021-05-24
Done with starting_2021-05-07
Done with starting_2021-05-09
Done with starting_2021-05-25
Done with starting_2021-05-29
Done with starting_2021-04-30
Done with starting_2021-05-28
Done with starting_2021-05-27
Done with starting_2021-05-06
Done with starting_2021-05-26
Done with starting_2021-05-08
Done with starting_2021-05-01
Done with starting_2021-05-05
Done with starting_2021-05-02
Done with starting_2021-05-04
Done with starting_2021-05-03
Done with starting_2021-05-30
Done with starting_2021-05-23
Done with starting_2021-05-10
Done with starting_2021-04-29
Finished in 3109.81 second(s)
